##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Transferencia de estilo artístico con TensorFlow Lite

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/lite/examples/style_transfer/overview"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/tensorflow/blob/master/tensorflow/lite/g3doc/examples/style_transfer/overview.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/tensorflow/blob/master/tensorflow/lite/g3doc/examples/style_transfer/overview.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a>
</td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/tensorflow/tensorflow/lite/g3doc/examples/style_transfer/overview.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar bloc de notas</a></td>
  <td>     <a href="https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2"><img src="https://www.tensorflow.org/images/hub_logo_32px.png">Ver modelo en TF Hub</a>
</td>
</table>

Uno de los avances más emocionantes en el aprendizaje profundo que ha aparecido recientemente es la [transferencia de estilo artístico](https://arxiv.org/abs/1508.06576), o su capacidad para crear una nueva imagen, conocida como [pastiche](https://en.wikipedia.org/wiki/Pastiche), a partir de dos imágenes de entrada: una que representa el estilo artístico y otra el contenido.

![Ejemplo de transferencia de estilo](https://storage.googleapis.com/download.tensorflow.org/models/tflite/arbitrary_style_transfer/formula.png)

Usando esta técnica, podemos generar nuevas y bellas obras de arte en una gama de estilos.

![Ejemplo de transferencia de estilo](https://storage.googleapis.com/download.tensorflow.org/models/tflite/arbitrary_style_transfer/table.png)

Si es nuevo en TensorFlow Lite y trabaja con Android, le recomendamos que explore las siguientes aplicaciones de ejemplo que pueden ayudarle a empezar.

<a class="button button-primary" href="https://github.com/tensorflow/examples/tree/master/lite/examples/style_transfer/android"> Ejemplo de Android </a><a class="button button-primary" href="https://github.com/tensorflow/examples/tree/master/lite/examples/style_transfer/ios"> Ejemplo de iOS</a>

Si utiliza una plataforma que no sea Android o iOS, o si ya está familiarizado con las <a href="https://www.tensorflow.org/api_docs/python/tf/lite">APIs de TensorFlow Lite</a>, puede seguir este tutorial para aprender a aplicar la transferencia de estilo en cualquier par de contenido e imagen de estilo con un modelo TensorFlow Lite preentrenado. Puede usar el modelo para añadir transferencia de estilo a sus propias aplicaciones móviles.

El modelo es de código abierto en [GitHub](https://github.com/tensorflow/magenta/tree/master/magenta/models/arbitrary_image_stylization#train-a-model-on-a-large-dataset-with-data-augmentation-to-run-on-mobile). Puede volver a entrenar el modelo con diferentes parámetros (por ejemplo, aumentar las ponderaciones de las capas de contenido para que la imagen de salida se parezca más a la imagen de contenido).

## Comprender la arquitectura del modelo

![Arquitectura del modelo](https://storage.googleapis.com/download.tensorflow.org/models/tflite/arbitrary_style_transfer/architecture.png)

Este modelo de transferencia de estilo artístico consta de dos submodelos:

1. **Modelo de predicción de estilo**: Una red neuronal basada en MobilenetV2 que convierte una imagen de estilo de entrada en un vector de cuello de botella de estilo de 100 dimensiones.
2. **Modelo de transformación de estilo**: Una red neuronal que aplica un vector de transformación de estilo a una imagen de contenido y crea una imagen estilizada.

Si su app sólo necesita admitir un conjunto fijo de imágenes de estilo, puede calcular de antemano sus vectores de cuello de botella de estilo y excluir el modelo de predicción de estilo del binario de su app.

## Configurar

Importe dependencias.

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
import IPython.display as display

import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (12,12)
mpl.rcParams['axes.grid'] = False

import numpy as np
import time
import functools

Descargue las imágenes de contenido y estilo, y los modelos TensorFlow Lite preentrenados.

In [ ]:
content_path = tf.keras.utils.get_file('belfry.jpg','https://storage.googleapis.com/khanhlvg-public.appspot.com/arbitrary-style-transfer/belfry-2611573_1280.jpg')
style_path = tf.keras.utils.get_file('style23.jpg','https://storage.googleapis.com/khanhlvg-public.appspot.com/arbitrary-style-transfer/style23.jpg')

style_predict_path = tf.keras.utils.get_file('style_predict.tflite', 'https://tfhub.dev/google/lite-model/magenta/arbitrary-image-stylization-v1-256/int8/prediction/1?lite-format=tflite')
style_transform_path = tf.keras.utils.get_file('style_transform.tflite', 'https://tfhub.dev/google/lite-model/magenta/arbitrary-image-stylization-v1-256/int8/transfer/1?lite-format=tflite')

## Preprocesar las entradas

- La imagen de contenido y la imagen de estilo deben ser imágenes RGB con valores de pixel que sean números float32 entre [0..1].
- El tamaño de la imagen de estilo debe ser (1, 256, 256, 3). Recortamos centralmente la imagen y cambiamos su tamaño.
- La imagen de contenido debe ser (1, 384, 384, 3). Recortamos centralmente la imagen y cambiamos su tamaño.

In [ ]:
# Function to load an image from a file, and add a batch dimension.
def load_img(path_to_img):
  img = tf.io.read_file(path_to_img)
  img = tf.io.decode_image(img, channels=3)
  img = tf.image.convert_image_dtype(img, tf.float32)
  img = img[tf.newaxis, :]

  return img

# Function to pre-process by resizing an central cropping it.
def preprocess_image(image, target_dim):
  # Resize the image so that the shorter dimension becomes 256px.
  shape = tf.cast(tf.shape(image)[1:-1], tf.float32)
  short_dim = min(shape)
  scale = target_dim / short_dim
  new_shape = tf.cast(shape * scale, tf.int32)
  image = tf.image.resize(image, new_shape)

  # Central crop the image.
  image = tf.image.resize_with_crop_or_pad(image, target_dim, target_dim)

  return image

# Load the input images.
content_image = load_img(content_path)
style_image = load_img(style_path)

# Preprocess the input images.
preprocessed_content_image = preprocess_image(content_image, 384)
preprocessed_style_image = preprocess_image(style_image, 256)

print('Style Image Shape:', preprocessed_style_image.shape)
print('Content Image Shape:', preprocessed_content_image.shape)

## Visualizar las entradas

In [ ]:
def imshow(image, title=None):
  if len(image.shape) > 3:
    image = tf.squeeze(image, axis=0)

  plt.imshow(image)
  if title:
    plt.title(title)

plt.subplot(1, 2, 1)
imshow(preprocessed_content_image, 'Content Image')

plt.subplot(1, 2, 2)
imshow(preprocessed_style_image, 'Style Image')

## Ejecutar la transferencia de estilo con TensorFlow Lite

### Predicción de estilo

In [ ]:
# Function to run style prediction on preprocessed style image.
def run_style_predict(preprocessed_style_image):
  # Load the model.
  interpreter = tf.lite.Interpreter(model_path=style_predict_path)

  # Set model input.
  interpreter.allocate_tensors()
  input_details = interpreter.get_input_details()
  interpreter.set_tensor(input_details[0]["index"], preprocessed_style_image)

  # Calculate style bottleneck.
  interpreter.invoke()
  style_bottleneck = interpreter.tensor(
      interpreter.get_output_details()[0]["index"]
      )()

  return style_bottleneck

# Calculate style bottleneck for the preprocessed style image.
style_bottleneck = run_style_predict(preprocessed_style_image)
print('Style Bottleneck Shape:', style_bottleneck.shape)

### Transformación de estilo

In [ ]:
# Run style transform on preprocessed style image
def run_style_transform(style_bottleneck, preprocessed_content_image):
  # Load the model.
  interpreter = tf.lite.Interpreter(model_path=style_transform_path)

  # Set model input.
  input_details = interpreter.get_input_details()
  interpreter.allocate_tensors()

  # Set model inputs.
  interpreter.set_tensor(input_details[0]["index"], preprocessed_content_image)
  interpreter.set_tensor(input_details[1]["index"], style_bottleneck)
  interpreter.invoke()

  # Transform content image.
  stylized_image = interpreter.tensor(
      interpreter.get_output_details()[0]["index"]
      )()

  return stylized_image

# Stylize the content image using the style bottleneck.
stylized_image = run_style_transform(style_bottleneck, preprocessed_content_image)

# Visualize the output.
imshow(stylized_image, 'Stylized Image')

### Mezcla de estilos

Podemos mezclar el estilo de la imagen de contenido en la salida estilizada, lo que a su vez hace que la salida se parezca más a la imagen de contenido.

In [ ]:
# Calculate style bottleneck of the content image.
style_bottleneck_content = run_style_predict(
    preprocess_image(content_image, 256)
    )

In [ ]:
# Define content blending ratio between [0..1].
# 0.0: 0% style extracts from content image.
# 1.0: 100% style extracted from content image.
content_blending_ratio = 0.5 #@param {type:"slider", min:0, max:1, step:0.01}

# Blend the style bottleneck of style image and content image
style_bottleneck_blended = content_blending_ratio * style_bottleneck_content \
                           + (1 - content_blending_ratio) * style_bottleneck

# Stylize the content image using the style bottleneck.
stylized_image_blended = run_style_transform(style_bottleneck_blended,
                                             preprocessed_content_image)

# Visualize the output.
imshow(stylized_image_blended, 'Blended Stylized Image')

## Puntos de referencia del rendimiento

Los números de referencia del rendimiento se generan con la herramienta [descrita aquí](https://www.tensorflow.org/lite/performance/benchmarks).

<table>
<thead>
<tr>
<th>Nombre del modelo</th> <th>Tamaño del modelo</th>  <th>Dispositivo</th> <th>NNAPI</th> <th>CPU</th> <th>GPU</th>
</tr> </thead>
<tr> <td rowspan="3"> <a href="https://tfhub.dev/google/lite-model/magenta/arbitrary-image-stylization-v1-256/int8/prediction/1?lite-format=tflite">Modelo de predicción de estilo (int8)</a>
</td>
<td rowspan="3">2.8 Mb</td>
<td>Pixel 3 (Android 10)</td> <td>142 ms</td>
<td>14 ms*</td>
<td></td>
</tr>
<tr>
<td>Pixel 4 (Android 10)</td> <td>5.2 ms</td>
<td>6.7 ms*</td>
<td></td>
</tr>
<tr>
<td>iPhone XS (iOS 12.4.1)</td> <td></td>
<td>10.7 ms**</td>
<td></td>
</tr>
<tr> <td rowspan="3"> <a href="https://tfhub.dev/google/lite-model/magenta/arbitrary-image-stylization-v1-256/int8/transfer/1?lite-format=tflite">Modelo de transformación de estilo (int8)</a>
</td>
<td rowspan="3">0.2 Mb</td>
<td>Pixel 3 (Android 10)</td> <td></td>
<td>540 ms*</td>
<td></td>
</tr>
<tr>
<td>Pixel 4 (Android 10)</td> <td></td>
<td>405 ms*</td>
<td></td>
</tr>
<tr>
<td>iPhone XS (iOS 12.4.1)</td> <td></td>
<td>251 ms**</td>
<td></td>
</tr>
<tr> <td rowspan="2"> <a href="https://tfhub.dev/google/lite-model/magenta/arbitrary-image-stylization-v1-256/fp16/prediction/1?lite-format=tflite">Modelo de predicción de estilo(float16)</a>
</td>
<td rowspan="2">4.7 Mb</td>
<td>Pixel 3 (Android 10)</td> <td>86 ms</td>
<td>28 ms*</td>
<td>9.1 ms</td>
</tr>
<tr>
<td>Pixel 4 (Android 10)</td>
<td>32 ms</td>
<td>12 ms*</td>
<td>10 ms</td>
</tr>
<tr> <td rowspan="2"> <a href="https://tfhub.dev/google/lite-model/magenta/arbitrary-image-stylization-v1-256/fp16/transfer/1?lite-format=tflite">Modelo de transferencia de estilo (float16)</a>
</td>
<td rowspan="2">0.4 Mb</td>
<td>Pixel 3 (Android 10)</td> <td>1095 ms</td>
<td>545 ms*</td>
<td>42 ms</td>
</tr>
<tr>
<td>Pixel 4 (Android 10)</td>
<td>603 ms</td>
<td>377 ms*</td>
<td>42 ms</td>
</tr>
</table>

** 4 hilos usados.*
*** 2 hilos en el iPhone para obtener el mejor rendimiento.*
